In [3]:
!pip install stable-baselines3[extra]

In [4]:
!pip install gym

In [5]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [6]:
environment = 'CartPole-v1'
env=gym.make(environment,render_mode="human")

In [8]:
epo=5
for episode in range(1, epo + 1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render() 
        action = env.action_space.sample()
        n_state, reward, done, _, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode, score))

env.close()


D:\stu\RL\rlenv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 1 Score: 23.0
Episode: 2 Score: 13.0
Episode: 3 Score: 10.0
Episode: 4 Score: 50.0
Episode: 5 Score: 14.0


In [9]:
env.action_space

Discrete(2)

In [10]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [11]:
env.observation_space.sample()

array([ 1.8647492e-01, -2.2643497e+38,  1.1770129e-02, -9.9035112e+37],
      dtype=float32)

In [12]:
env.action_space.sample()

0

train

In [45]:
log = os.path.join('training','log')

In [14]:
log

'traning\\log'

In [15]:
!pip install shimmy>=2.0

In [38]:
env=gym.make(environment,render_mode="human")
env= DummyVecEnv([lambda:env])
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log)

Using cpu device


In [22]:
PPO??

Init signature:
PPO(
    policy: Union[str, Type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gymnasium.core.Env, ForwardRef('VecEnv'), str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    normalize_advantage: bool = True,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    rollout_buffer_class: Optional[Type[stable_baselines3.common.buffers.RolloutBuffer]] = None,
    rollout_buffer_kwargs: Optional[Dict[str, Any]] = None,
    target_kl: Optional[float] = None,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[Dict[str, Any]]

In [24]:
model.learn(total_timesteps=20000)

Logging to traning\log\PPO_2
-----------------------------
| time/              |      |
|    fps             | 46   |
|    iterations      | 1    |
|    time_elapsed    | 43   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 45           |
|    iterations           | 2            |
|    time_elapsed         | 89           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0062728687 |
|    clip_fraction        | 0.0397       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.535       |
|    explained_variance   | 0.814        |
|    learning_rate        | 0.0003       |
|    loss                 | 2.81         |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.00615     |
|    value_loss           | 22.2         |
------------------------------

In [26]:
model_path=os.path.join('training','save_models','ppoCartPole')

In [28]:
model.save(model_path)

In [29]:
del model

In [39]:
model = PPO.load(model_path,env=env)

In [40]:
model.learn(total_timesteps=100)

Logging to traning\log\PPO_2
-----------------------------
| time/              |      |
|    fps             | 46   |
|    iterations      | 1    |
|    time_elapsed    | 44   |
|    total_timesteps | 2048 |
-----------------------------


In [41]:
evaluate_policy(model,env,n_eval_episodes=10,render=True)

(500.0, 0.0)

In [33]:
env.close()

In [44]:
epo=3
for episode in range(1, epo + 1):
    obs = env.reset()
    done = False
    score = 0
    while not done:
        env.render() 
        action,_= model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode, score))

Episode: 1 Score: [500.]
Episode: 2 Score: [500.]
Episode: 3 Score: [500.]


In [46]:
tLogPath =os.path.join(log,'PPO_2')

In [47]:
tLogPath

'training\\log\\PPO_2'

In [48]:
!pip install tensorboard

In [50]:
!tensorboard --logdir={tLogPath}#go to local host 6006

^C


In [54]:
from stable_baselines3.common.callbacks import EvalCallback as evc,StopTrainingOnRewardThreshold as strt

In [55]:
save=os.path.join('training','saved_models')

In [69]:
stop = strt(reward_threshold=200,verbose=1)
eval = evc(env,
           callback_on_new_best=stop,
           eval_freq=4000,
           best_model_save_path=save,
            verbose=1)

In [61]:
model =PPO('MlpPolicy',env,verbose=1,tensorboard_log=log)

Using cpu device


In [62]:
model.learn(total_timesteps=20000,callback=eval)

Logging to training\log\PPO_3
-----------------------------
| time/              |      |
|    fps             | 45   |
|    iterations      | 1    |
|    time_elapsed    | 45   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 45          |
|    iterations           | 2           |
|    time_elapsed         | 89          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009929149 |
|    clip_fraction        | 0.123       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.685      |
|    explained_variance   | -0.00129    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.93        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0191     |
|    value_loss           | 46.9        |
-----------------------------------------
----

In [66]:
net_arch=[dict(pi=[128,128,128,128],
              vf=[128,128,128,128])]

In [67]:
model =PPO('MlpPolicy',env,verbose=1,tensorboard_log=log,policy_kwargs={'net_arch':net_arch})

Using cpu device


In [70]:
model.learn(total_timesteps=20000,callback=eval)

Logging to training\log\PPO_5
-----------------------------
| time/              |      |
|    fps             | 46   |
|    iterations      | 1    |
|    time_elapsed    | 44   |
|    total_timesteps | 2048 |
-----------------------------
Eval num_timesteps=4000, episode_reward=325.00 +/- 146.22
Episode length: 325.00 +/- 146.22
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 325         |
|    mean_reward          | 325         |
| time/                   |             |
|    total_timesteps      | 4000        |
| train/                  |             |
|    approx_kl            | 0.018314738 |
|    clip_fraction        | 0.214       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.646      |
|    explained_variance   | 0.416       |
|    learning_rate        | 0.0003      |
|    loss                 | 12.2        |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.0352   

In [71]:
from stable_baselines3 import DQN

In [72]:
model =DQN('MlpPolicy',env,verbose=1,tensorboard_log=log)

Using cpu device


In [73]:
model.learn(total_timesteps=20000)

Logging to training\log\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.959    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 46       |
|    time_elapsed     | 1        |
|    total_timesteps  | 87       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.931    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 45       |
|    time_elapsed     | 3        |
|    total_timesteps  | 146      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.533    |
|    n_updates        | 11       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.874    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 4